Part 0:

DONE

Part 1:

DONE

Part 2:

DONE (see lines 50-63)

Part 3:

DONE (using orthogonal initialization - see lines 93-113).  I found that uniform initializatoin caused NaN values boooo!
![alt text](https://thleats-bucket.s3.us-east-2.amazonaws.com/CS/Lab3_v2.JPG)




Part 4:

Quiz: DONE

Using a Kernel size of 3×3 what should the settings of your 2d convolution be that results in the following mappings (first answer given to you)

    (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : (out_channels=10, kernel_size=(3, 3), padding=(0, 0))
    (c=3, h=10, w=10) ⇒ (c=22, h=10, w=10) : **(out_channels=22, kernel_size=(3,3), padding=(1,1))**
    (c=3, h=10, w=10) ⇒ (c=65, h=12, w=12) : **(out_channels=65, kernel_size=(3,3), padding=(2,2))**
    (c=3, h=10, w=10) ⇒ (c=7, h=20, w=20) : **(out_channels=7, kernel_size=(3,3), padding=(6,6))**

Using a Kernel size of 5×5:)

    (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : (out_channels=10, kernel_size=(5, 5), padding=(1, 1))
    (c=3, h=10, w=10) ⇒ (c=100, h=10, w=10) : **(out_channels=100, kernel_size=(5,5), padding=(2,2))**
    (c=3, h=10, w=10) ⇒ (c=23, h=12, w=12) : **(out_channels=23, kernel_size=(5,5), padding=(3,3))**
    (c=3, h=10, w=10) ⇒ (c=5, h=24, w=24) : **(out_channels=5, kernel_size=(5,5), padding=(9,9))**

Using Kernel size of 5×3:

    (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : **(out_channels=10, kernel_size=(5,3), padding=(1,0))**
    (c=3, h=10, w=10) ⇒ (c=100, h=10, w=10) : **(out_channels=100, kernel_size=(5,3), padding=(2,1))**
    (c=3, h=10, w=10) ⇒ (c=23, h=12, w=12) : **(out_channels=23, kernel_size=(5,3), padding=(3,2))**
    (c=3, h=10, w=10) ⇒ (c=5, h=24, w=24) : **(out_channels=5, kernel_size=(5,3), padding=(9,8))**

Determine the kernel that requires the smallest padding size to make the following mappings possible:

    (c=3, h=10, w=10) ⇒ (c=10, h=9, w=7) :**(out_channels=10, kernel_size=(2,4), padding=(0,0))**
    (c=3, h=10, w=10) ⇒ (c=22, h=10, w=10) : **(out_channels=22, kernel_size=(1,1), padding=(0,0))**

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 13 12:09:11 2019

@author: Nolan
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
import scipy
from scipy import signal
from torch.nn.parameter import Parameter
import pdb
 
assert torch.cuda.is_available() # You need to request a GPU from Runtime > Change Runtime Type

def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            total_param += num_param
    return total_param
  

class Conv2d(nn.Module):
  def __init__(self, n_channels,out_channels,kernel_size,init_strategy='xav',stride=1,padding=0,dilation=1,groups=1,bias=True):
    self.__dict__.update(locals())
    super(Conv2d,self).__init__()
    self.weight = Parameter(torch.Tensor(self.out_channels,
                                         self.n_channels,
                                         kernel_size[0],
                                        kernel_size[1]))
    #pdb.set_trace()
    self.bias = Parameter(torch.Tensor(out_channels))
    
    #self.weight.data.uniform_(-1,1)
    self.bias.data.uniform_(0,0)
    
    ##########################Initialization Strategy#######################
    
    #init_strategy = 'xav'
    
    if init_strategy == 'xav':
      torch.nn.init.xavier_uniform_(self.weight)
    elif init_strategy == 'orth':
      M=np.random.random((self.out_channels,self.n_channels * kernel_size[0] * kernel_size[1])).astype(np.float32)
      U,_, Vt= np.linalg.svd(M,full_matrices=False)
      if len(M)>len(M[0]):
        W=U.reshape((self.out_channels,self.n_channels,kernel_size[0],kernel_size[1]))
      else:
        W=Vt.reshape((self.out_channels,self.n_channels,kernel_size[0],kernel_size[1]))
      self.weight.data=torch.from_numpy(W)
    elif init_strategy == 'uni':
      self.weight.data.uniform_(-1,1)
    else:
      torch.nn.init.xavier_uniform_(self.weight)
    ########################################################################

  def forward(self,x):
    return F.conv2d(x,self.weight,self.bias,self.stride,
                    self.padding,self.dilation,self.groups)

class CE_me(nn.Module):
  def __init__(self,weight=None,size_average=None,ignore_index=-100,reduce=None,reduction='mean'):
    super(CE_me,self).__init__()
    
    
  def forward(self,y_hat,y_truth):
    right_side = torch.log(torch.sum(torch.exp(y_hat),dim=1))
    n,classes = y_hat.size()
    b=torch.zeros((n,classes))
    b[np.arange(n),y_truth]=1
    left_side = torch.sum(y_hat * b.cuda(),dim=1)
    return torch.mean(-left_side + right_side)
    

########################Convolution Network Class###############################
parameters=[]
class ConvNetwork(nn.Module):
  def __init__(self):
    super(ConvNetwork,self).__init__()
    c=1
    out = 10
    
    self.net = nn.Sequential(
        Conv2d(c,10,(3,3),init_strategy='orth',padding=(1,1)),
        nn.ReLU(),
        
        Conv2d(10,64,(3,3),init_strategy='orth',padding=(1,1)),        
        nn.ReLU(),
        
        Conv2d(64,64,(3,3),init_strategy='orth',padding=(1,1)),        
        nn.ReLU(),
        
        Conv2d(64,64,(5,5),init_strategy='orth',padding=(2,2)),        
        nn.ReLU(),
        
        Conv2d(64,64,(5,5),init_strategy='orth',padding=(2,2)),        
        nn.ReLU(),
        
        Conv2d(64,64,(7,7),init_strategy='orth',padding=(3,3)),        
        nn.ReLU(),
        

        
        Conv2d(64,64,(9,9),init_strategy='orth',padding=(4,4)),        
        nn.ReLU(),
              
        
        Conv2d(64,10,(28,28), padding = (0,0)),
        nn.ReLU(),
        
        
        
        Conv2d(10,out,(1,1), padding=(0,0)),
     
        
    )
    
  def forward(self,x):
    return self.net(x).squeeze(2).squeeze(2)
########################################################################    
        
class FashionMNISTProcessedDataset(Dataset):
  def __init__(self,root,train=True):
    self.data=datasets.FashionMNIST(root,train=train,transform=transforms.ToTensor(),download=True)   
  def __getitem__(self,i):
    x,y=self.data[i]
    return x,y
  def __len__(self):
    return len(self.data)
    
train_dataset=FashionMNISTProcessedDataset('/tmp/fashionmnist',train=True)
val_dataset=FashionMNISTProcessedDataset('/tmp/fashionmnist',train=False)

model = ConvNetwork()
model=model.cuda()
#model = LinearNetwork(train_dataset)

objective = CE_me()#replace this


optimizer = optim.Adam(model.parameters(),lr=1e-4)
train_loader = DataLoader(train_dataset,
                         batch_size=42,
                         pin_memory=True)
valid_load = DataLoader(val_dataset,batch_size=round(len(val_dataset)/len(train_loader)),pin_memory=True)


losses=[]
losses_valid=[]
loss_mean=[]
acc_vec=[]
acc_epoch=[]
epochs=10
accuracy_v=0
accuracy_t=0
acc_vec_t=[]
for epoch in range(epochs):
  
  loop = tqdm(total=len(train_loader), position = 0)
  
  for loader in zip(train_loader,valid_load):

    x,x_valid = [thing[0] for thing in loader]
    y_truth,y_valid = [thing[1] for thing in loader]

    x,y_truth = x.cuda(async=True),y_truth.cuda(async=True)
    
    #x_valid,y_truth_valid = x_valid.cuda(async=True),y_valid.cuda(async=True)
    
    optimizer.zero_grad()
    
    
    
    #with torch.no_grad():
     # y_nohat=model(x_valid)
      #loss_valid = objective(y_nohat,y_truth_valid)
      
    y_hat=model(x)
    
    pre_acc_t = torch.argmax(y_hat,1)-y_truth
    accuracy_t = (y_truth.size(0)-pre_acc_t.nonzero().size(0))/pre_acc_t.size(0)
    pdb.set_trace()
    
    loss = objective(y_hat, y_truth)
    loss.backward()
    optimizer.step()

    losses.append(loss)
    #losses_valid.append(loss_valid)
 
    loop.set_description('loss:{:.4f}'.format(loss.item()))    
    loop.update(1)
    acc_vec_t.append(accuracy_t)
  
  accuracy_v=0
  for x_valid,y_truth_valid in valid_load:
    x_valid,y_truth_valid = x_valid.cuda(async=True),y_truth_valid.cuda(async=True)
    
    with torch.no_grad():
      y_nohat=model(x_valid)
      pre_acc = torch.argmax(y_nohat,1)-y_truth_valid
      accuracy_v = (y_truth_valid.size(0)-pre_acc.nonzero().size(0))/pre_acc.size(0)
      
      #pdb.set_trace()
      loss_valid = (objective(y_nohat,y_truth_valid))
      losses_valid.append(loss_valid.item())
    loss_mean.append(np.mean(losses_valid))
    acc_vec.append(accuracy_v)
  
  loop.close()
print(len(losses))

iterations=range(0,len(losses),1)
valid_iterations=range(0,len(loss_mean),1)
#losses_valid=signal.savgol_filter(losses_valid,51,2)

losses=signal.savgol_filter(losses,21,2)
acc_vec=signal.savgol_filter(acc_vec,21,2)
acc_vec_t=signal.savgol_filter(acc_vec_t,21,2)

fix, ax = plt.subplots()

loss_v=ax.plot(iterations,loss_mean,label='Validation Loss')
loss_t=ax.plot(iterations,losses,label='Training Loss')
acc=ax.plot(iterations,acc_vec,label='Accuracy Validation')
acc_t=ax.plot(iterations,acc_vec_t,label='Accuracy Training')



ax.legend()


ax.set(xlabel='iterations', ylabel='loss & accuracy',
       title='Lab2 Plot')
ax.grid()


plt.show()


print('Total parameters is: ' + str(count_parameters(model)))

from google.colab import drive
drive.mount('/content/drive')
with open('/home/ubuntu/foo.txt', 'w') as f:
  out=torch.save(model.state_dict(),'/content/drive/My Drive/foo.txt')
  
files.download('foo.txt')


  0%|          | 0/1429 [00:00<?, ?it/s]

> <ipython-input-1-39bfa4f0c7d0>(192)<module>()
-> loss = objective(y_hat, y_truth)


In [0]:
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  out=torch.save(model.state_dict(),'/content/drive/My Drive/foo.txt')
  
files.download('foo.txt')
